<a href="https://colab.research.google.com/github/aaryaaa16/Healthwise/blob/main/symptom_checker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install datasets

In [ ]:
from datasets import load_dataset
import json

dataset = load_dataset("shanover/disease_symptoms_prec_full")

data_list = dataset["train"]
data_list = data_list.to_pandas().to_dict(orient="records")

with open("dataset.json", "w", encoding="utf-8") as json_file:
    json.dump(data_list, json_file, ensure_ascii=False, indent=2)

In [ ]:
import json
from datasets import load_dataset

dataset = load_dataset("shanover/disease_symptoms_prec_full")

data_list = dataset["train"]
data_list = data_list.to_pandas().to_dict(orient="records")

squad_format = {
    "data": []
}

processed_diseases = set()

for record in data_list:
    disease = record.get("disease", "")
    symptoms = record.get("symptoms", "")
    precautions = record.get("precautions", "")

    if not disease or not symptoms or not precautions:
        continue

    if disease in processed_diseases:
        continue

    processed_diseases.add(disease)

    context = f"{disease} is a medical condition that can cause {symptoms}."

    # qa_pair = {
    #     "context": context,
    #     "qas": [
    #         {
    #             "question": f"If someone has the symptoms {symptoms}, what is the possible disease?",
    #             "id": len(squad_format["data"]) + 1,
    #             "answers": [
    #                 {
    #                     "text": disease,
    #                     "answer_start": context.find(disease)
    #                 }
    #             ]
    #         },
    #         {
    #             "question": "What precautions should be taken for this disease?",
    #             "id": len(squad_format["data"]) + 2,
    #             "answers": [
    #                 {
    #                     "text": precautions,
    #                     "answer_start": context.find(precautions)
    #                 }
    #             ]
    #         }
    #     ]
    # }

    squad_format["data"].append({
        # "title": disease,
        "context": context,
        # "paragraphs": [qa_pair]
        "qas": [
            {
                "question": f"If someone has the symptoms {symptoms}, what is the possible disease?",
                "id": len(squad_format["data"]) + 1,
                "answers": [
                    {
                        "text": disease,
                        "answer_start": context.find(disease)
                    }
                ]
            },
            # {
            #     "question": "What precautions should be taken for this disease?",
            #     "id": len(squad_format["data"]) + 2,
            #     "answers": [
            #         {
            #             "text": precautions,
            #             "answer_start": context.find(precautions)
            #         }
            #     ]
            # }
        ]
    })

with open("dataset_squad.json", "w", encoding="utf-8") as json_file:
    json.dump(squad_format, json_file, ensure_ascii=False, indent=2)


In [ ]:
! pip install --upgrade transformers

In [ ]:
import json
with open(r"dataset_squad.json", "r") as read_file:
  train = json.load(read_file)

In [ ]:
train

{'data': [{'context': 'Fungal infection is a medical condition that can cause itching,skin_rash,nodal_skin_eruptions,dischromic__patches.',
   'qas': [{'question': 'If someone has the symptoms itching,skin_rash,nodal_skin_eruptions,dischromic__patches, what is the possible disease?',
     'id': 1,
     'answers': [{'text': 'Fungal infection', 'answer_start': 0}]}]},
  {'context': 'GERD is a medical condition that can cause stomach_pain,acidity,ulcers_on_tongue,vomiting,cough,chest_pain.',
   'qas': [{'question': 'If someone has the symptoms stomach_pain,acidity,ulcers_on_tongue,vomiting,cough,chest_pain, what is the possible disease?',
     'id': 2,
     'answers': [{'text': 'GERD', 'answer_start': 0}]}]},
  {'context': 'Chronic cholestasis is a medical condition that can cause itching,vomiting,yellowish_skin,nausea,loss_of_appetite,abdominal_pain,yellowing_of_eyes.',
   'qas': [{'question': 'If someone has the symptoms itching,vomiting,yellowish_skin,nausea,loss_of_appetite,abdominal_

In [ ]:
import os

os.listdir()


['.config',
 'dataset.json',
 'outputs',
 'runs',
 'cache_dir',
 'dataset_squad.json',
 'sample_data']

In [ ]:
!pip install simpletransformers

In [ ]:
import json
with open(r"dataset_squad.json", "r") as read_file:
    train = json.load(read_file)

In [ ]:
import logging

from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

In [ ]:

model_type="bert"
model_name= "bert-base-cased"
if model_type == "bert":
    model_name = "bert-base-cased"

elif model_type == "roberta":
    model_name = "roberta-base"

elif model_type == "distilbert":
    model_name = "distilbert-base-cased"

elif model_type == "distilroberta":
    model_type = "roberta"
    model_name = "distilroberta-base"

elif model_type == "electra-base":
    model_type = "electra"
    model_name = "google/electra-base-discriminator"

elif model_type == "electra-small":
    model_type = "electra"
    model_name = "google/electra-small-discriminator"

elif model_type == "xlnet":
    model_name = "xlnet-base-cased"

In [ ]:
model_args = QuestionAnsweringArgs()
model_args.train_batch_size = 16
model_args.n_best_size = 3
model_args.num_train_epochs = 5

In [ ]:
train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "output_dir": f"outputs/{model_type}",
    "best_model_dir": f"outputs/{model_type}/best_model",
    "max_seq_length": 128,
    "num_train_epochs": 5,
    "save_model_every_epoch": False,
    "train_batch_size": 128,
    "eval_batch_size": 64,
}

In [ ]:
model = QuestionAnsweringModel(
    model_type, model_name, args=train_args, use_cuda=False
)


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.train_model(train['data'])



convert squad examples to features:   0%|          | 0/38 [00:00<?, ?it/s]

convert squad examples to features: 100%|██████████| 38/38 [00:00<00:00, 219.95it/s]


add example index and unique id: 100%|██████████| 38/38 [00:00<00:00, 182779.30it/s]


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 1 of 5:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 2 of 5:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 3 of 5:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/1 [00:00<?, ?it/s]

(5, 4.320829010009765)

In [ ]:
result, texts = model.eval_model(train["data"])



convert squad examples to features:   0%|          | 0/38 [00:00<?, ?it/s]

convert squad examples to features: 100%|██████████| 38/38 [00:00<00:00, 44.72it/s]


add example index and unique id: 100%|██████████| 38/38 [00:00<00:00, 149095.93it/s]


Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
result

{'correct': 33, 'similar': 4, 'incorrect': 1, 'eval_loss': -0.6920462250709534}

In [ ]:
texts

{'correct_text': {1: 'Fungal infection',
  2: 'GERD',
  3: 'Chronic cholestasis',
  4: 'Peptic ulcer diseae',
  5: 'AIDS',
  6: 'Diabetes ',
  7: 'Gastroenteritis',
  8: 'Bronchial Asthma',
  9: 'Hypertension ',
  10: 'Migraine',
  11: 'Cervical spondylosis',
  13: 'Jaundice',
  14: 'Malaria',
  16: 'Dengue',
  17: 'Typhoid',
  18: 'hepatitis A',
  19: 'Hepatitis B',
  20: 'Hepatitis C',
  21: 'Hepatitis D',
  22: 'Hepatitis E',
  23: 'Alcoholic hepatitis',
  24: 'Tuberculosis',
  26: 'Pneumonia',
  28: 'Varicose veins',
  29: 'Hypothyroidism',
  30: 'Hyperthyroidism',
  31: 'Hypoglycemia',
  32: 'Osteoarthristis',
  33: 'Arthritis',
  35: 'Acne',
  36: 'Urinary tract infection',
  37: 'Psoriasis',
  38: 'Impetigo'},
 'similar_text': {12: {'truth': 'Paralysis (brain hemorrhage)',
   'predicted': 'Paralysis',
   'question': 'If someone has the symptoms vomiting,headache,weakness_of_one_body_side,altered_sensorium, what is the possible disease?'},
  15: {'truth': 'Chicken pox',
   'predi

In [ ]:
to_predict = [
    {
        "context": "Chronic cholestasis is a medical condition that can cause itching,vomiting,yellowish_skin,nausea,loss_of_appetite,abdominal_pain,yellowing_of_eyes.",
        "qas": [
            {
                "question": "If someone has the symptoms itching,vomiting,yellowish_skin,nausea,loss_of_appetite,abdominal_pain,yellowing_of_eyes, what is the possible disease?",
                "id": "0",
            }
        ],
    }
]

In [ ]:
answers, probabilities = model.predict(to_predict)

predicted_disease = answers[0]['answer'][0]

output = f"Predicted disease: {predicted_disease}"

print(output)




convert squad examples to features: 100%|██████████| 1/1 [00:00<00:00, 61.14it/s]


add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 7133.17it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

Predicted disease: Chronic cholestasis
